<a href="https://colab.research.google.com/github/djvaroli/CS542/blob/master/airbnb_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
full_df = pd.read_csv("data.csv")
features = list(full_df)

test_ind = pd.read_csv("test.csv")
train_ind = pd.read_csv("train.csv")
val_ind = pd.read_csv("val.csv")

train_df = full_df.merge(train_ind, on=["id"])
val_df = full_df.merge(val_ind, on=["id"])
test_df = full_df.merge(test_ind, on=["id"])



In [0]:
def predict_price(new_listing_value,feature_column,k = 5):
    temp_df = train_df
    temp_df['distance'] = np.abs(train_df[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn = temp_df.price.iloc[:k]
    predicted_price = knn.mean()
    return predicted_price

In [0]:
prices = test_df.accommodates.apply(predict_price,feature_column='accommodates')
